In [311]:
import os

from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.VrmlAPI import VrmlAPI_Writer
from OCC.Core.VrmlData import VrmlData_WorldInfo
from OCC.Core.XCAFDoc import XCAFDoc_ShapeTool, XCAFDoc_DocumentTool, XCAFDoc_DocumentTool_ShapeTool
from OCC.Core.XCAFApp import XCAFApp_Application_GetApplication
from OCC.Core.XCAFDoc import XCAFDoc_DocumentTool_ShapeTool, xcafdoc
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TDF import TDF_LabelSequence, TDF_Label, TDF_ChildIterator, TDF_AttributeIterator, TDF_Tool, TDF_AttributeMap, TDF_Attribute, TDF_ChildIDIterator
from OCC.Core.TCollection import TCollection_AsciiString
from OCC.Core.TDataStd import TDataStd_TreeNode
from OCC.Core.Standard import Standard_GUID #, Standard_Handle
#from OCC.Core.Standard import Standard_Real 
from OCC.Core.TNaming import TNaming_UsedShapes
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Extend.TopologyUtils import is_edge, is_wire, discretize_edge, discretize_wire, get_type_as_string
from OCC.Extend.DataExchange import read_step_file_with_names_colors, read_step_file
from OCC.Display.WebGl.x3dom_renderer import X3DExporter, X3DomRenderer, export_edge_to_indexed_lineset
from OCC.Core.Tesselator import ShapeTesselator
from OCC.Core.TopAbs import topabs
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.gp import gp_XYZ, gp_Vec
from OCC.Core.TopoDS import TopoDS_Builder
from OCC.Core.BRep import BRep_Builder
from OCC.Core.TopoDS import TopoDS_Solid, TopoDS_Shell

import xml.etree.ElementTree as ET
from IPython.display import HTML
import x3d.x3d as XX3D
import re
import copy


In [376]:
#stp_filename = 'Ventilator'#AP203' #'as1_pe_203'
stp_filename = 'as1-oc-214'
#stp_filename = 'as1_pe_203'
#stp_filename = 'screw'
#stp_filename = 'RC_Buggy_2_front_suspension'
#stp_filename = 'KR600_R2830-4'
stp_path = os.path.join('assets', stp_filename + '.stp')

# load the STEP file
doc = TDocStd_Document(TCollection_ExtendedString("pythonocc-doc"))

step_reader = STEPCAFControl_Reader()
step_reader.SetColorMode(True)
step_reader.SetLayerMode(True)
step_reader.SetNameMode(True)
step_reader.SetMatMode(True)
step_reader.SetGDTMode(True)

status = step_reader.ReadFile(stp_path)
if status == IFSelect_RetDone:
    step_reader.Transfer(doc)
else:
	raise IOError("STEP file could not be read.")

In [377]:
##Copyright 2018 Thomas Paviot (tpaviot@gmail.com)
##(C) 2020 Andreas Plesch
##
##This file is part of pythonOCC.
##
##pythonOCC is free software: you can redistribute it and/or modify
##it under the terms of the GNU Lesser General Public License as published by
##the Free Software Foundation, either version 3 of the License, or
##(at your option) any later version.
##
##pythonOCC is distributed in the hope that it will be useful,
##but WITHOUT ANY WARRANTY; without even the implied warranty of
##MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
##GNU Lesser General Public License for more details.
##
##You should have received a copy of the GNU Lesser General Public License
##along with pythonOCC.  If not, see <http://www.gnu.org/licenses/>.

import os

from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.StlAPI import stlapi_Read, StlAPI_Writer
from OCC.Core.BRep import BRep_Builder
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pnt2d
from OCC.Core.Bnd import Bnd_Box2d
from OCC.Core.TopoDS import TopoDS_Compound
from OCC.Core.IGESControl import IGESControl_Reader, IGESControl_Writer
from OCC.Core.STEPControl import STEPControl_Reader, STEPControl_Writer, STEPControl_AsIs
from OCC.Core.Interface import Interface_Static_SetCVal
from OCC.Core.IFSelect import IFSelect_RetDone, IFSelect_ItemsByEntity
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.XCAFDoc import (XCAFDoc_DocumentTool_ShapeTool,
                              XCAFDoc_DocumentTool_ColorTool)
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Core.TDF import TDF_LabelSequence, TDF_Label
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

from OCC.Extend.TopologyUtils import (discretize_edge, get_sorted_hlr_edges,
                                      list_of_shapes_to_compound)

try:
    import svgwrite
    HAVE_SVGWRITE = True
except ImportError:
    HAVE_SVGWRITE = False
    
def scenegraph_from_document_with_names_colors(doc, log=False):
    """ Returns a scene as list of node dictionaries
    Uses OCAF.
    """
    output_shapes = {}
    scene = []
    visited = {}
    DEFset = {}
    facesInSubshapes = set()

    shape_tool = XCAFDoc_DocumentTool_ShapeTool(doc.Main())
    color_tool = XCAFDoc_DocumentTool_ColorTool(doc.Main())
    
    locs = []
    
    def print_log( message ):
        if (log):
            print (message)
        return
    
    def _get_sub_shapes(lab, loc, parent):
        
        labelString = lab.EntryDumpToString()
        if labelString in visited:
            return
        
        visited[labelString] = lab
        name = _unescapeStep(lab.GetLabelName())
        
        if shape_tool.IsAssembly(lab):
            node = {
                    'node' : 'Group',
                    'DEF' : labelString,
                    'name' : name,
                    'children' : []
            }
            l_c = TDF_LabelSequence()
            shape_tool.GetComponents(lab, l_c)
            for i in range(l_c.Length()):
                label = l_c.Value(i + 1)
                #print("Group Name DEF :", name, labelString)    
                if shape_tool.IsReference(label):
                    print_log("########  component label :"+ _unescapeStep(label.GetLabelName()) )
                    loc = shape_tool.GetLocation(label)
                    #print(" Transform  loc DEF          :", loc.HashCode(100))
                    label_reference = TDF_Label()
                    shape_tool.GetReferredShape(label, label_reference)
                    reference_name = _unescapeStep(label_reference.GetLabelName())
                    print_log("########  Transform USE to DEF ==> referenced label : "+ reference_name )
                    trafo = {
                        'node' : 'Transform',
                        'DEF' : label.EntryDumpToString(),
                        'name' : reference_name + '-trafo',
                        'transform' : loc,
                        'children': []
                    }                
                    reference_entry = label_reference.EntryDumpToString()
                    if reference_entry in DEFset: #already defined, use USE
#                         reference = {
#                             'node' : 'Transform',
#                             'USE' : reference_entry,
#                             'name' : reference_name + '-ref'
#                         }

#                        trafo['children'].append(reference)
                        trafo['children'].append(DEFset[reference_entry]) # pointer to DEF
                        # x3d generation now checks for reuse and applies USE as required
                    else:
                        #DEFset[reference_entry] = node # moved since simple shapes also need to be pointed to
                        _get_sub_shapes(label_reference, loc, trafo['children'])
                    node['children'].append(trafo)
                    
        elif shape_tool.IsSimpleShape(lab): # TODO recursive dive
            #print("Transform DEF Shape Name :", name, labelString )
            shape = shape_tool.GetShape(lab)
            shape_type = get_type_as_string(shape)
            print_log(" #######  simpleshape of type " + shape_type + " for : " + name)
            
            c = _set_color(lab, shape)
            clabel = color_tool.FindColor(c)
            clabelString = clabel.EntryDumpToString()
            
            n = c.Name(c.Red(), c.Green(), c.Blue())
            #print('    instance color Name & RGB: ', n, c.Red(), c.Green(), c.Blue())
            
            labloc = shape_tool.GetLocation(lab)
            #print("    Shape Transform: ", labloc.HashCode(100))
     
            ##subshapes
            l_subss = TDF_LabelSequence()
            shape_tool.GetSubShapes(lab, l_subss)
            
            #node = {}
            
            if (l_subss.Length() == 0 and labloc.IsIdentity()): # does not need transform
                # but still needs transform wrapper for proper USE reference
                node = {
                    'node' : 'Transform',
                    'DEF' : labelString,
                    'name' : name + '-wrapper',
                    'children' : []
                }
                
                shapenode = {
                    'node' : 'Shape',
                    'DEF' : labelString+'-shape',
                    'label' : lab,
                    'shape' : shape,
                    'shapeType' : shape_type,
                    'name' : name + '-shape',
                    'colorString' : f"{c.Red()} {c.Green()} {c.Blue()}",
                    'color' : (c.Red(), c.Green(), c.Blue()),
                    'colorDEF' : clabelString
                }
                
                node['children'].append(shapenode)

            else: # needs grouping or has transform
                node = {
                    'node' : 'Transform',
                    'DEF' : labelString,
                    'transform' : labloc,
                    'transformhash' : labloc.HashCode(100),
                    'name' : name,
                    'children' : []
                }

                shapenode = {
                    'node' : 'Shape',
                    'DEF' : labelString+'-shape',
                    'label' : lab,
                    'shape' : shape,
                    'shapeType' : shape_type,
                    'name' : name + '-shape',
                    'colorString' : f"{c.Red()} {c.Green()} {c.Blue()}",
                    'color' : (c.Red(), c.Green(), c.Blue()),
                    'colorDEF' : clabelString
                }
                
                if (l_subss.Length() == 0): # only attach if no subshapes
                    #print("no subshapes: "+shape_type+" for :"+name)
                    node['children'].append(shapenode)
            
            for i in range(l_subss.Length()):
                
                lab_subs = l_subss.Value(i+1)
                shape_sub = shape_tool.GetShape(lab_subs)
                shape_type = get_type_as_string(shape_sub)
                #print("########  simpleshape subshape of type "+shape_type+" for :", name)
                #l_subsubss = TDF_LabelSequence()
                #shape_tool.GetSubShapes(lab_subs, l_subsubss)
                #print("########  subshape has subshapes: " + str(l_subsubss.Length()))
                #print("########  subshape has faces: ", len(solidfaces))
                #print("########  subshape has shells: ", expl.number_of_shells())
                
                c = _set_color(lab_subs, shape_sub)
                clabel = color_tool.FindColor(c)
                clabelString = clabel.EntryDumpToString()
                n = c.Name(c.Red(), c.Green(), c.Blue())
                #print('    solidshape color RGB: ', c.Red(), c.Green(), c.Blue(), n)
                node_name = _unescapeStep(lab_subs.GetLabelName())
                def_name = lab_subs.EntryDumpToString() # should always exist
                subloc = shape_tool.GetLocation(lab_subs) # assume identity, otherwise we need another wrapper
                #print("    subshape Transform: ", subloc.HashCode(100))
                #default subshape
                shapenode = {
                    'node' : 'SubShape',
                    'DEF' : def_name,
                    'label' : lab_subs,
                    'shape' : shape_sub,
                    'shapeType' : shape_type,
                    'DEF' : def_name,
                    'name' : node_name + '-subshape',
                    'colorString' : f"{c.Red()} {c.Green()} {c.Blue()}",
                    'color' : (c.Red(), c.Green(), c.Blue()),
                    'colorDEF' : clabelString,
                    'trafo' : subloc                        
                    }
                
                ### look for face colors
                expl = TopologyExplorer(shape_sub)
                solidfaces=list( expl.faces() ) # works for all types
                #hasMultiColor = False
                if ( len(solidfaces) > 0 ):
                    colorFaceLists = {}
                    colorColors = {}
                    #print ("shapetype of solidface: ", get_type_as_string(solidfaces[0]))
                    facelabel = TDF_Label()
                    for i in range(0, len(solidfaces)):
                        found = shape_tool.FindSubShape(lab, solidfaces[i], facelabel)
                        if (found):
                            facesInSubshapes.add(facelabel.EntryDumpToString())
                            c = _set_color(facelabel, solidfaces[i])
                            clabel = color_tool.FindColor(c)
                            clabelString = clabel.EntryDumpToString()
                            if clabelString not in colorColors:
                                colorFaceLists[clabelString] = []
                            colorFaceLists[clabelString].append(solidfaces[i]) # collect face
                            colorColors[clabelString] = c # collect color
                    
                    # override default color, if only one color, is last color
                    clabel = color_tool.FindColor(c)
                    clabelString = clabel.EntryDumpToString()
                    shapenode['colorString'] = f"{c.Red()} {c.Green()} {c.Blue()}"
                    shapenode['color'] = (c.Red(), c.Green(), c.Blue())
                    shapenode['colorDEF'] = clabelString
                    
#                     for entry in iter(colorColors):
#                         c2 = colorColors[entry]
#                         print('    solidface color RGB: ', entry, c2.Red(), c2.Green(), c2.Blue())
                        
                    # if more colors, make group with a shell per color (or compounds ?)
                    if ( len(list(colorFaceLists)) > 1 ):
                        
                        shapenode = {
                            'node' : 'Group',
                            'label' : lab_subs,
                            'shape' : shape_sub,
                            'shapeType' : shape_type,
                            'DEF' : def_name,
                            'name' : node_name+'-group',
                            'children' : []
                        }                
                        f = 0
                        for entry in iter(colorFaceLists):
                            #make new shell with faces of single color
                            builder =  BRep_Builder()
                            subshell = TopoDS_Shell() #use compound ?
                            builder.MakeShell(subshell)
                            faces = colorFaceLists[entry]
                            for i in range(0, len(faces)): # add faces
                                #print(entry,i,colorLists[entry][i])
                                builder.Add(subshell, faces[i])
                            #shape_type = get_type_as_string(subshell)
                            shape_type = "Shell"
                            c = colorColors[entry]
                            clabel = color_tool.FindColor(c)
                            clabelString = clabel.EntryDumpToString()
                            shellnode = {
                                'node' : 'SubShape',
                                'label' : lab_subs,
                                'shape' : subshell,
                                'shapeType' : shape_type,
                                'DEF' : f"{def_name}:{f}",
                                'name' : node_name+'-colorshell',
                                'colorString' : f"{c.Red()} {c.Green()} {c.Blue()}",
                                'color' : (c.Red(), c.Green(), c.Blue()),
                                'colorDEF' : clabelString
                            }
                            shapenode['children'].append(shellnode) #  add to group
                            f = f + 1
                        #hasMultiColor = True
                    #//end grouping into single color
                #//end face color check
                
                node['children'].append(shapenode)
            #//end subshapes
        DEFset[node['DEF']] = node
        parent.append(node)

    def _set_color(lab, shape):
        #rint('is visible: ',color_tool.IsVisible(lab))
        c = Quantity_Color(0.5, 0.5, 0.5, Quantity_TOC_RGB)  # default color
        colorSet = False
        if (color_tool.GetInstanceColor(shape, 0, c) or
            color_tool.GetInstanceColor(shape, 1, c) or
            color_tool.GetInstanceColor(shape, 2, c)):

            colorSet = True

        if not colorSet:
            if (color_tool.GetColor(lab, 0, c) or
                color_tool.GetColor(lab, 2, c) or
                color_tool.GetColor(lab, 1, c)):

                colorSet = True

        if colorSet:
            color_tool.SetInstanceColor(shape, 0, c)
            color_tool.SetInstanceColor(shape, 1, c)
            color_tool.SetInstanceColor(shape, 2, c)
        
        return c
    
    def _unescapeStep(name):
        
        def _toUnicode(match):
            return chr(int(match.group(1), 16))
        
        reg1 = re.compile(r'\\X\\(..)')
        reg2 = re.compile(r'\\X2\\(....)\\X0\\')
        reg3 = re.compile(r'\\X4\\(........)\\X0\\')
        
        return reg3.sub(_toUnicode, 
                        reg2.sub(_toUnicode, 
                                 reg1.sub(_toUnicode, name)))
        
    def _get_shapes():
        labels = TDF_LabelSequence()
        shape_tool.GetFreeShapes(labels)
        
        print_log("Number of shapes at root : " + str(labels.Length()))
        
        for i in range(labels.Length()):
            root_item = labels.Value(i+1)
            _get_sub_shapes(root_item, None, scene)

    _get_shapes()
    print_log('DONE')
    return { 'scenegraph': scene, 'internalFaceEntrySet': facesInSubshapes }

def x3d_from_scenegraph(scene=[], facesInSolids=None, show_edges=True, edge_color=(0,0,0), log=False):#, doc=None):
    
    #shape_tool = XCAFDoc_DocumentTool_ShapeTool(doc.Main())
    
    DEFset = {}
    
    if facesInSolids is None:
        facesInSolids = set()
        
    def print_log( message ):
        if (log):
            print ( "X3D: "+str(message) )
        return
    
    def _x3dapplyLocation(x3dtransformnode, location):
            # get translation and rotation from location
            transformation = location.Transformation()
            rot_axis = gp_XYZ()
            #rot_angle = 0.0
            success, rot_angle = transformation.GetRotation(rot_axis)#.GetVectorAndAngle(rot_axis, rot_angle)
            translation = transformation.TranslationPart()
            scale_factor = transformation.ScaleFactor()
            x3dtransformnode.rotation = (rot_axis.X(), rot_axis.Y(), rot_axis.Z(), rot_angle)
            x3dtransformnode.translation = (translation.X(), translation.Y(), translation.Z())
            x3dtransformnode.scale = (scale_factor, scale_factor, scale_factor)
            return

    def _x3dgeofromTShape(shape):
        geo = XX3D.Box()
        #if label.IsNull():
        #    return geo
        #shape = shape_tool.GetShape(label)
        tesselator = ShapeTesselator(shape)
        tesselator.Compute(
            compute_edges=True,
            mesh_quality=1,
            parallel=True)
        nbr_edges = tesselator.ObjGetEdgeCount() # should be just one
        edge_point_set = []
        vertexCountList = []
        for i_edge in range(nbr_edges):
            nbr_vertices = tesselator.ObjEdgeGetVertexCount(i_edge)
            for i_vert in range(nbr_vertices):
                edge_point_set.append(tesselator.GetEdgeVertex(i_edge, i_vert))
            vertexCountList.append(nbr_vertices)
        mf_points = []
        for p in edge_point_set:
            mf_points.append( (p[0], p[1], p[2]) )
        coord = XX3D.Coordinate(point = mf_points)
        edges = XX3D.LineSet(coord = coord, vertexCount = vertexCountList) # add DEF
        if ( is_edge(shape) or is_wire(shape) ):
            geo = edges
            edges = None
        else:
            x3dstring = tesselator.ExportShapeToX3DIndexedFaceSet()#x3dexp._triangle_sets[0] # there should be just one
            element = ET.XML(x3dstring)
            geo = XX3D.Box()
            if (element.tag == 'TriangleSet'):
                coordele = list(element.iter('Coordinate'))[0]
                mf_points = _MFVec3ffromString(coordele.attrib['point'])
                # can be empty
                if (len(mf_points) > 0):
                    coord = XX3D.Coordinate(point = mf_points)
                    normalele = list(element.iter('Normal'))[0]
                    mf_vectors = _MFVec3ffromString(normalele.attrib['vector'])
                    normal = XX3D.Normal(vector = mf_vectors)
                    geo = XX3D.TriangleSet(coord = coord, normal=normal, solid=False) # add DEF
            # get tesselated triangleset or lineset
        return { 'x3dgeo': geo, 'x3dedges': edges }
    
    def _MFVec3ffromString(sepString):
        mflist = sepString.split()
        mf = []
        for i in range(len(mflist)):
            if (i % 3 == 2):
                mf.append((float(mflist[i-2]), float(mflist[i-1]), float(mflist[i])))
        return mf
    
    def _x3dappfromColor(c, DEFname, emissive):
        if (emissive):
            DEFname = DEFname + "-emissive"
        if DEFname in DEFset:
            DEFset[DEFname] = DEFset[DEFname] + 1 # counter not used
            print_log('app reused: ' + DEFname)
            return XX3D.Appearance(USE = DEFname)
        else:
            DEFset[DEFname] = 0
            if (emissive):
                x3dmat = XX3D.Material(emissiveColor = c)
            else:
                x3dmat = XX3D.Material(diffuseColor = c, specularColor = (0.9, 0.9, 0.9), shininess = 1, ambientIntensity = 0.1)
            return XX3D.Appearance(DEF = DEFname, material = x3dmat)
    
    def _getx3dnode(node):
        
        def _sanitizeDEF(name):
# IdFirstChar ::=
# Any ISO-10646 character encoded using UTF-8 except: 0x30-0x3a, 0x0-0x20, 0x22, 0x23, 0x27, 0x2b, 0x2c, 0x2d, 0x2e, 0x5b, 0x5c, 0x5d, 0x7b, 0x7d, 0x7f ;
# first no [0-9],space,",#,',+,comma,-,.,[,\,],{,}
# IdRestChars ::=
# Any number of ISO-10646 characters except: 0x0-0x20, 0x22, 0x23, 0x27, 0x2c, 0x2e, 0x3a, 0x5b, 0x5c, 0x5d, 0x7b, 0x7d, 0x7f ;
# rest no space,",#,',comma,.,:,[,\,],{,}
            return 'L-' + ( name
                          .replace(" ","_")
                          .replace('"','^')
                          .replace('#','N')
                          .replace("'","^")
                          .replace(",",";")
                          .replace(".",";")
                          .replace(":","-")
                          .replace("[","(")
                          .replace("]",")")
                          .replace("{","(")
                          .replace("}",")")
                          .replace("\\","/") )

        def _applyDEFUSE(x3dnode=None, suffix=None): # suffix for edges
            if (suffix == None):
                suffix = ''
            if 'USE' in node:
                x3dnode.USE = _sanitizeDEF(node['USE'])
            if 'DEF' in node:
                # now DEFs need to be checked for previous use
                DEFname = _sanitizeDEF(node['DEF']) + str(suffix)
                if DEFname in DEFset:
                    DEFset[DEFname] = DEFset[DEFname] + 1 #count number of uses
                    x3dnode.USE = DEFname
                    print_log('  reusing ' + DEFname + ": " + str(DEFset[DEFname]))
                else:
                    DEFset[DEFname] = 0
                    x3dnode.DEF = DEFname
                    print_log('  defining ' + x3dnode.DEF)
            return
        
        def _cleanUSEgeo(geometry):
            if (geometry.USE is not None):
                geometry.coord = None
                geometry.normal = None
        
        def _applyDEF(x3dnode, suffix=None):
            if (suffix == None):
                suffix = ''
            if 'USE' in node:
                print_log('Unexpected USE key in node:' + str(node))
                #x3dnode.USE = _sanitizeDEF(node['USE'])
            if 'DEF' in node:
                # now DEFs need to be checked for previous use
                DEFname = _sanitizeDEF(node['DEF']) + str(suffix)
                if DEFname in DEFset:
                    DEFset[DEFname] = DEFset[DEFname] + 1 #count number of uses
                    x3dnode.DEF = DEFname + "-ref-" + str( DEFset[DEFname] )
                    print_log("  redefined " + DEFname + " as " + x3dnode.DEF)
                else:
                    DEFset[DEFname] = 0
                    x3dnode.DEF = DEFname
                    print_log("  defining " + x3dnode.DEF)
            return
        
        def _applychildren(x3dnode):
            if 'children' in node:
                for n in node['children']:
                    success, childlist = _getx3dnode(n)
                    if success:
                        x3dnode.children.extend(childlist)
            return
        
        def _appendTouchLogic(nodes, switch_DEF):
            ts_DEF = x3dnode.DEF + '-ts'
            brancher_DEF = x3dnode.DEF + '-brancher'
            triggerONE_DEF = x3dnode.DEF + '-triggerOne'
            triggerZERO_DEF = x3dnode.DEF + '-triggerZero'
            
            ts = XX3D.TouchSensor(DEF = ts_DEF, description = x3dnode.class_ + '-sensor')
            brancher = XX3D.BooleanFilter( DEF = brancher_DEF )
            triggerone = XX3D.IntegerTrigger ( DEF = triggerONE_DEF, integerKey = 1 )
            triggerzero = XX3D.IntegerTrigger ( DEF = triggerZERO_DEF, integerKey = 0 )
        
            route1 = XX3D.ROUTE(fromNode = ts_DEF, fromField = 'isOver',
                                toNode = brancher_DEF, toField = 'set_boolean')
            route2 = XX3D.ROUTE(fromNode = brancher_DEF, fromField = 'inputTrue', 
                                toNode = triggerONE_DEF, toField = 'set_boolean')
            route3 = XX3D.ROUTE(fromNode = brancher_DEF, fromField = 'inputFalse', 
                                toNode = triggerZERO_DEF, toField = 'set_boolean')
            route4 = XX3D.ROUTE(fromNode = triggerONE_DEF, fromField = 'triggerValue', 
                                toNode = switch_DEF, toField = 'whichChoice')
            route5 = XX3D.ROUTE(fromNode = triggerZERO_DEF, fromField = 'triggerValue', 
                                toNode = switch_DEF, toField = 'whichChoice')
            
            nodes.extend([ts, brancher, triggerone, triggerzero, route1, route2, route3, route4, route5])
            return
        
        if not 'node' in node:
            print_log('no node type, skipping')
            return False, None
        
        if 'DEF' in node:
            if node['DEF'] in facesInSolids:
                #print('in skipped list')
                return False, None
        
        ntype = node['node']
        siblingnodes = []
        
        if (ntype == 'Group'):
            x3dnode = XX3D.Group(class_ = node['name'], children = [])
            _applyDEF(x3dnode)
            _applychildren(x3dnode)
        
        elif (ntype == 'Transform'):
            x3dnode = XX3D.Transform(class_ = node['name'], children = [])
            if 'transform' in node:
                _x3dapplyLocation(x3dnode, node['transform'])
            _applyDEF(x3dnode)
            _applychildren(x3dnode)
        
        elif (ntype == 'Shape' or ntype == 'SubShape'):
            shape_type = node['shapeType']
#             if (shape_type != "Solid" and shape_type != "Shell"):
#                 return False, None
            if 'shape' in node:
                print_log("adding shape: " + str(node))
                x3dnode = XX3D.Switch(class_ = node['name'], whichChoice = 0, children = [])
                _applyDEF (x3dnode)
                
                defaultgroup = XX3D.Group (class_ = x3dnode.class_ + '-regular', children = [])
                x3dnode.children.append(defaultgroup)
                _applyDEF (defaultgroup, '-regular')
                #defaultgroup.DEF = defaultgroup.DEF + '-regular' # seems out of place
                facenode = XX3D.Shape (class_ = x3dnode.class_ + '-main')
                _applyDEF(facenode, '-main')
                defaultgroup.children.append(facenode)
                print_log("adding main (face or edge or wire)")
                shape = node['shape']
                isEdgeOrWire = is_edge(shape) or is_wire(shape)
                if (not isEdgeOrWire):
                    edgenode = XX3D.Shape (class_ = x3dnode.class_ + '-edge', visible = show_edges)    
                    _applyDEF(edgenode, '-edge')
                    defaultgroup.children.append(edgenode)
                    print_log("adding edge")
                #print_log(x3dnode.XML())
                if (facenode.USE is None): # and edgenode.USE is None): #should parallel
                    geometryDict = _x3dgeofromTShape(shape)
                    facenode.geometry = geometryDict['x3dgeo']
                    _applyDEFUSE(facenode.geometry, '-main-geo')
                    _cleanUSEgeo(facenode.geometry)
                    if 'color' in node:
                        colorDEF = _sanitizeDEF(node['colorDEF'])
                        facenode.appearance = _x3dappfromColor(node['color'], colorDEF, isEdgeOrWire)
                    if (not isEdgeOrWire):
                        edgenode.geometry = geometryDict['x3dedges']
                        _applyDEFUSE(edgenode.geometry, '-edge-geo')
                        _cleanUSEgeo(edgenode.geometry)
                        edgenode.appearance = _x3dappfromColor(edge_color, 'app-faceedge', True)
                
                #pick group
                pickgroup = XX3D.Group (class_ = x3dnode.class_ + '-pick', children = [])
                x3dnode.children.append(pickgroup)
                _applyDEF (pickgroup, '-pick')
                #pickgroup.DEF = pickgroup.DEF + '-pick' # seems out of place
                facenode = XX3D.Shape (class_ = x3dnode.class_ + '-main-pick')
                _applyDEF(facenode, '-main-pick')
                pickgroup.children.append(facenode)
                print_log("adding main-pick (face or edge or wire)")
                shape = node['shape']
                isEdgeOrWire = is_edge(shape) or is_wire(shape)
                if (not isEdgeOrWire):
                    edgenode = XX3D.Shape (class_ = x3dnode.class_ + '-edge-pick', visible = show_edges)    
                    _applyDEF(edgenode, '-edge-pick')
                    pickgroup.children.append(edgenode)
                    print_log("adding edge")
                #print_log(x3dnode.XML())
                if (facenode.USE is None): # and edgenode.USE is None): #should parallel
                    geometryDict = _x3dgeofromTShape(shape)
                    facenode.geometry = geometryDict['x3dgeo']
                    _applyDEFUSE(facenode.geometry, '-main-geo-pick')
                    _cleanUSEgeo(facenode.geometry)
                    #if 'color' in node:
                    #    colorDEF = _sanitizeDEF(node['colorDEF'])
                    facenode.appearance = _x3dappfromColor((1, 1, 1), 'pick-white', isEdgeOrWire)
                    if (not isEdgeOrWire):
                        edgenode.geometry = geometryDict['x3dedges']
                        _applyDEFUSE(edgenode.geometry, '-edge-geo-pick')
                        _cleanUSEgeo(edgenode.geometry)
                        edgenode.appearance = _x3dappfromColor((1, 0, 0), 'pick-app-faceedge', True)
                
                #add logic
                _appendTouchLogic(siblingnodes, x3dnode.DEF)
                
                
        else:
            print_log ('unknown node: --' + ntype + "--")
        
        x3dnodelist = [x3dnode]
        x3dnodelist.extend(siblingnodes)
        return True, x3dnodelist

    x3dscene = XX3D.Scene(children=[])
    x3ddoc = XX3D.X3D(Scene = x3dscene)
    
    for node in scene:
        success, x3dnodelist = _getx3dnode(node)
        if success:
            x3dscene.children.extend(x3dnodelist)
        else:
            print_log ('no x3d for root node, skipped')
    
    return x3ddoc

def x3d_doc_from_document(doc=None, show_edges=True, edge_color=(0, 0, 0), log=False):
    s = scenegraph_from_document_with_names_colors(doc, log)
    return x3d_from_scenegraph(s['scenegraph'], s['internalFaceEntrySet'], show_edges, edge_color, log)


In [383]:
#ascene, skipfaces=scenegraph_from_document_with_names_colors(doc)
x3ddoc=x3d_doc_from_document(doc, True, (0, 1, 1))#, True)

In [384]:
#print(x3ddoc.XML())

In [385]:
x3domHEAD = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script> 
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>'''

In [386]:
x3dele = list(ET.XML(x3ddoc.XML()).iter('X3D'))[0]
next(x3dele.iter('Scene')).append(ET.XML('<Environment gammaCorrectionDefault="none" frustumCulling="false" />'))
x3dHTML = ET.tostring(x3dele, encoding="unicode", short_empty_elements=False)
x3dHTML = x3dHTML.replace("visible=",'render=') # x3dom uses render
#print(x3dHTML)

In [387]:
HTML(x3domHEAD + x3dHTML)